In [ ]:
# Standard Imports
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, random, sys
import matplotlib.dates as mdates
import pandas as pd

In [ ]:
def MakeDataFromTxt(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif len(line)==8:               # If it is a time stamp
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s))) # Minus 3 for Pacific Time
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

In [ ]:
t1 = MakeDataFromTxt('tags_40003.csv')

# Combine all data into one dictionary
combined = dict()
for d in [ t1, ]:
    for k,v in d.items():
        if k in combined:                                 # if tag is already in dict
            for i in range(len(v[0])):                    # iterate over all time points
                if v[0][i] in combined[k]:                # if time point is already in dict
                    combined[k][v[0][i]].append(v[1][i])
                else:                                     # if time point is not in dict
                    combined[k][v[0][i]] = [v[1][i]]
        else:             # if tag is not in dict
            combined[k] = dict()
            for i in range(len(v[0])):
                combined[k][v[0][i]] = [v[1][i]]
for k,v in combined.items():
    for k2,v2 in v.items():
        combined[k][k2] = sum(v2)/len(v2) # avg all points taken during the same second

In [ ]:
shirt1 = ["61DF", "2BC6", "427A", "0F7D", "8099", "759A", "9A80", "7198", "94B4", "988C", "80D2", "79D3", "8265", "896D", "7852", "4E7A",]
glove1 = ["6091", "147F", "2AC0", "637B", "43C3", "83B0", "63D4", "3B5C",]
pants1 = ["409D", "1734", "A389", "AC37", "5439", "508F", "67A1", "701B",]
sock1L =["B19F", "653F", "7D30",]
beanie1 = ["624F", "3E6B"]

glove2 = ["A457", "2C4A", "91AA", "2693", "780A", "AD41", "4E4C", "3B61"]
glove3 = ["BA65", "6344", "A998", "62A5", "3697", "6E17", "3AAE", "A891"]
glove4 = ["6953", "1776", "4C65", "9AB2", "369D", "5B3C", "52C9", "4F0C"]
glove5 = ["3598", "9722", "123C", "9A48", "2E8E", "9C80", "2E9B", "B1A2"]
glove6 = ["4050", "3E5D", "9DB4", "2E3B", "822E", "AF99", "9326", "7CC2"]
glove7 = ["9E31", "A959", "949B", "42D2", "7596", "69BC", "4697", "7C53"]
glove8 = ["8432", "91D2", "885A", "4B37", "45A4", "A060", "8AC9", "4F98"]

shirt2 = ["B57A", "088C", "8853", "5471", "3DBA", "0709", "9E3B", "8F77", "6065", "5782", "9B7D", "0A4A", "A831", "813D", "077C", "A035"]
shirt3 =["1591", "8C56", "66C9", "1364", "6D2E", "0B61", "1E72", "1B7E", "5C34", "2371", "5E8F", "3DCF", "92C1", "265C", "7181", "2774" ]
shirt4 =["6FD7", "62CA", "A73D", "056D", "4743", "6ACF", "37BB", "1650", "2CC8", "5713", "7EB3", "2C98", "3150", "424E", "4FDC", "B0A2" ]
shirt5 = ["0086", "3644", "1E9E", "3325", "099B", "5577", "2D1C", "5352", "855B", "3B75", "1CBB", "7CD4", "2A7A", "3C47", "8E8D", "9941"]
shirt6 = ["B4A6", "138F", "41C1", "6A9E", "AB82", "98C0", "067B", "734B", "41AC", "1C2E", "8938", "B142", "A38C", "8FA4", "7BDA", "8917"]
shirt7 = ["9353", "7E20", "2E4F", "60D1", "7B06", "4EE3", "3318", "6415", "0D7C", "1EB3", "8A72", "2D69", "1F90", "77DE", "2850", "9F6B"]
shirt8 =["277D", "77B5", "0859", "431C", "3869", "833E", "774F", "2B4F", "64A5", "483D", "0E92", "AC75", "675C", "737F", "2567",  "B76E"]

pants2 = ["5E5E", "AB37", "8992", "9873", "846F", "592A", "8220", "6227"]
pants3 = ["6BA1", "4465", "9063", "2445", "1D74", "3A8A", "726C", "8C46"]
pants4 = ["2AAB", "113D", "7646", "1336", "1885", "6BB0", "842A", "2C4E"]
pants5 = ["299E", "0887", "B35D", "41B6", "60A1", "8960", "A296", "6CBE"]
pants6 = ["028E", "472E", "2853", "B64D", "016B", "38CD", "8045", "268E"]
pants7 = ["1572", "83BB", "8895", "8B48", "8175", "1EA6", "297A", "239D"]
pants8 = ["877A", "1D35", "1838", "8060", "3E0C", "64D4", "5F1A", "64D4"]

sock1R = ["9B83", "9850", "B27A"]
sock2L = ["4C48", "7E4B", "824B"]
sock2R = ["3932", "5372", "4B8B"]
sock3L =["2EC8", "4187", "2B26" ]
sock3R = ["4D16", "5D86", "6F91"]
sock4L = ["55BA", "713D", "7244"]
sock4R = ["5C1B", "2C29", "A95E"]

beanie2 =  ["3B3C", "0962" ]
beanie3 = ["83A7", "7195"  ]
beanie4 =  ["6D29", "3159" ]

In [ ]:
if False:
    list_dfs = list()

    for k,v in combined.items():          # For each EPC in all the readings
        # if k[-4:] in epcs:                # If we have the right EPC
            s = sorted( v.items() )
            time, data = [ j[0] for j in s ], [ j[1] for j in s ]

            sdf = pd.DataFrame(
                data = { 'Datetime':time, k[-4:]:data },
                columns=[ 'Datetime', k[-4:] ]
            )
            list_dfs.append(sdf)

    all_data = pd.concat([t for t in list_dfs], ignore_index=False)
    all_data = all_data.set_index('Datetime')
    all_data = all_data.sort_index()
    all_data = all_data.groupby(level=0).sum()
    all_data = all_data.groupby(all_data.index).sum()
    all_data = all_data.replace(0, "-")
    all_data.to_excel("tags_40003_Excel.xlsx")

In [ ]:
# PLOTTING CODE
fig, ax = plt.subplots(figsize=(11, 6.5))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

# shirt8 =["277D", "77B5", "0859", "431C", "3869", "833E", "774F", "2B4F", "64A5", "483D", "0E92", "AC75", "675C", "737F", "2567",  "B76E"]
# glove2 = ["A457", "2C4A", "91AA", "2693", "780A", "AD41", "4E4C", "3B61"]

shirt8 = {
    "277D" : "red",
    "77B5" : "blue",
    "0859" : "forestgreen",
    "431C" : "c",
    "3869" : "darkorange",
    "833E" : "springgreen",
    "774F" : "darkmagenta",
    "2B4F" : "black",
    "64A5" : "#fe0f6a",
    "483D" : "red",
    "0E92" : "blue",
    "AC75" : "forestgreen",
    "675C" : "c",
    "737F" : "darkorange",
    "2567" : "springgreen",
    "B76E" : "darkmagenta",

    "A457" : 'red',
    "2C4A" : 'blue',
    "91AA" : 'forestgreen',
    "2693" : 'c',
    "780A" : 'darkorange',
    "AD41" : 'springgreen',
    "4E4C" : 'darkmagenta',
    "3B61" : 'black',
}

label = {
    "277D" : "Left Subscap Out",
    "77B5" : "Left Subscap In",
    "0859" : "Right Subscap Out",
    "431C" : "Right Subscap In",
    "3869" : "Chest In",
    "833E" : "Right Upper Arm In",
    "774F" : "Left Upper Arm In",
    "2B4F" : "Right Lower Arm In",
    "64A5" : "Left Lower Arm In",
    "483D" : "Abdomen In",
    "0E92" : "Chest Out",
    "AC75" : "Right Upper Arm Out",
    "675C" : "Left Upper Arm Out",
    "737F" : "Right Lower Arm Out",
    "2567" : "Left Lower Arm Out",
    "B76E" : "Abdomen Out",

    "A457" : "Left Thigh In",
    "2C4A" : "Left Thigh Out",
    "91AA" : "Right Thigh Out",
    "2693" : "Right Thigh In",
    "780A" : "Left Calf In",
    "AD41" : "Left Calf Out",
    "4E4C" : "Right Calf Out",
    "3B61" : "Right Calf In",
}

for k,v in combined.items():
    if k[-4:] in [ "780A", "AD41"  ]:  
        s = sorted( v.items() )
        time, data = [ j[0] for j in s ], [ j[1] for j in s ]

        ax.plot( time, data, label=label[k[-4:]], color=shirt8[k[-4:]], linewidth=0.95, marker="." )


ax.legend()
ax.set_title("Left Calf In Readings from tags_40003")
# ax.set_ylim(19, 25.5)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend( by_label.values(), by_label.keys() )